In [1]:
#Key
#Notes- Paramters contain underscores, while functions don't
#SB: Stay at bower
#FG: Foraging
#MT: Maraud travel (travel to rival's bower)
#MA: Maraud action (destroy bower if absent, have antagonistic interaction in present)
#MR: Maraud return (return to own bower)
#RB: Repair own bower
#MT_vs_FG: the probability of next transition away from SB being to MT (otherwise it would be to FG)
#a=-1: denotes a staying at bower action on ticket
#a=-2: denotes a foraging action on ticket
#a=-3: denotes a female visiting action on a ticket
#a=-4: denotes travel to another male's bower
#a=-5: denotes marauding of another male's bower
#a=-6: denotes returning from another male's bower
#a=-7: denotes a bower repair action on a ticket
#target: the male/node an action is preformed on. For maurading, the target is the bird under potential attack. For mating, target is male being mated with. For other activities other then maurading and mating, owner and target are the same.
#owner: The male or female actively performing an action. For mating, the female is the owner. For all other activities, the owner is male.
#networkwriter: generates network where everybird is connected
#addtotimeline: adds tickets to timeline
#travel_times_linear_p: generates a matrix that contains the probabilities of travelling to a given a bower 
#if the relationship between distance and travel preference is linear
#improb: The probability of traveling improb_distance or less
#improb_distance: The distance at which there is only a (1-improb)% chance of choosing to travel
#lamb: lambda calculated by solving improb=1-numpy.exp(-lamb*improb_distance)
#bower_states: options are 1 (bower intact) and 0 (bower destroyed-will never be the case when all guarders) and another value for time that mating will end (assuming mating is happening)
#male_states: options are [leave_time, leave_action, SB_timeholder] (male present at bower) and 0 (male absent from bower)
#fitness_states: keeps track of number of matings a given male has had
#t: time
#nodes: total number of male bowerbirds in the network
#node_dist: the node by nde matrix with distances between each node
#node_graph: matrix with individuals 1 through nodes numbered by going through each row
#tau: Length of time an activity takes. If a foraging ticket has a tau of 0.5 and occurs at t=1, then the bird is foraging for 30 minutes, at which point a SB activity will start (at t=1.5).
#t_court: the amount of time it takes a female to court+mate with a male, generated from a normal distribution
#not_maraud_time: the time added to SB by guarders


import math
import random
import numpy as numpy
import matplotlib.pyplot as plt


#Parameters:
t=0.0 #start at time 0
t_max=12*30 #2 months in hrs (assume no nights so 12 hr = 1 day)
timeholder=0
timeline=[] #initialize the timeline
discards=[] #where we put the old version of tickets that get over-written/moved
#We've moved away from using these params with expovariate (how we handle FV is still TBD)
#So THE BELOW 3 SHOULD BE COMMENTED OUT (but rn we don't want the handler funcs to yell at us)
FV_param=[0, t_max] #5days, tmax/3 #0.2 #nodes/0.19 #totally random
FG_param=3 #larger values lead to foraging ending earlier, because taus are smaller
SB_param=1.5 #larger values lead to bowerstay ending earlier, because taus are smaller
RB_param=2 #made up -- takes 30mins (every time -- no distribution) to repair bower
MA_param=.1 #made up -- takes 6mins (every time -- no distribution) to maraud bower
MT_vs_FG=.1 #.05 originally #compared 20 feedings/day to .1 maraudings/hr
nodes=4
bower_states=[1]*nodes
male_states=[0]*nodes #start at foraging now
mating_states=[0]*nodes
fitness_states=[0]*nodes
success_rate=1
success_times=[] #going to change this to be a vector of times when the female leaves (at this point, sucessful mating has occured)
recents_list=[]
max_visits=6 #STEEEEVEEEE
bird_speed=12*3600 #m/hr (12m/s)
d=100.0
FperM=3 #number of sexually mature females per sexually mature males in a bowerbird population
num_sims=100
not_maraud_time=.15

def ticketgenerator(tau,t, o, a, targ, t2):
    ticket={
        'tau': tau, #how long the activity lasts
        'time': t, #time at which the activity starts
        'owner': o, #who is doing the activity (could be a female if action=-3)
        'action': a, #which activity (see key)
        'target': targ, #target of the activity (owner=target unless action=-3,-4,-5, or -6)
        'origin': t2 #the time of the ticket that generated this one
    }
    return ticket;

#adds new tickets to timeline -- Stefano, any more efficient suggestions for keeping timeline in order?
def addtotimeline(tic): #A CHANGE
    if not timeline:
        timeline.append(tic)
    else:
        ind=len(timeline)-1
        end=0
        while (tic['time']<timeline[ind]['time'] and end==0): #moves backwards until it finds where to place the ticket based on the listed times
            ind=ind-1 
            if(ind<0):
                end=1
        ind=ind+1
        timeline.insert(ind, tic)


#travel time
#distance function
def travel(nodes,d,bird_speed):
    node_dist= numpy.array([[-1.0]*nodes]*nodes) #(dist in m)initialize a nodes-by-nodes matrix (1st nrows, 2nd ncols)
    sqrt_nodes=int(math.sqrt(nodes)) 
    node_graph=numpy.arange(nodes)
    node_mat=node_graph.reshape(sqrt_nodes,sqrt_nodes)
    for i in range(sqrt_nodes):
        for j in range (sqrt_nodes):
            n1=node_mat[i][j]
            for a in range(sqrt_nodes):
                for b in range(sqrt_nodes):
                    n2=node_mat[a][b]
                    if n1<n2:
                        d12=math.hypot((i-a)*d,abs(j-b)*d)
                        node_dist[n1][n2]=d12
                        node_dist[n2][n1]=d12
                    if n1==n2:
                        node_dist[n1][n2]=0
    travel_times=numpy.array([[0.0]*nodes]*nodes)
    for i in range(nodes):
        for j in range(nodes):
            travel_times[i][j]=node_dist[i][j]/bird_speed
    return [node_dist,travel_times, node_mat]




travel_mats=travel(nodes,d,bird_speed)
travel_times=travel_mats[1]
print(travel_times)



# function for determining the next time based on our rate parameters
def nexttau(action, ow, targ, new_t):
    new_tau=-1
    while new_tau<=0:
        switcher = { #NOTE: used abs
            -1: numpy.random.normal(loc=.1583, scale=.09755, size=1)[0], #choose when to leave bower (generate a tau for bower stay)
            -2: numpy.random.gamma(shape=1.5, scale=5, size=1)[0]/60, #choose when to stop foraging (generate a tau for foraging)
            -3: numpy.random.uniform(FV_param[0], FV_param[1]), #FV_param... totally arbitrary so we should think about it
            -4: travel_times[ow][targ],
            -5: MA_param, #in the future we'll do something with it
            -6: travel_times[targ][ow],
            -7: RB_param #in the future we'll do something with it 
        }
        new_tau=switcher.get(action)
    future_t=new_tau+new_t #time on future ticket will be the current time plus the tau
    if future_t>t_max: 
        new_tau=new_tau-(future_t-t_max) #this ensures that the tau+t won't exceed t_max
    return new_tau

def futurebuilder(old_tic, new_ac, new_targ):
    old_t=old_tic['time']
    new_t=old_t+old_tic['tau']
    new_tic={}
    if new_t<t_max:
        ow=old_tic['owner']
        new_tic=ticketgenerator(nexttau(new_ac, ow, new_targ, new_t), new_t, ow, new_ac, new_targ, old_t)
        addtotimeline(new_tic)
    return new_tic
    
    
#solve for lambda 
improb=0.99
improb_distance=800
lamb=-math.log(1-improb)/improb_distance

#will write female preference based on cumulative exponential decay (lambda=.00576)
def femaleprobs(node_dist, nodes, lamb):
    visit_preferences=numpy.array([[0.]*nodes]*nodes)
    for i in range(nodes):
        for j in range(nodes):
            if i!=j:
                visit_preferences[i][j]=math.exp(-lamb*node_dist[i][j])        
        visit_preferences[i]=visit_preferences[i]/sum(visit_preferences[i])
    return visit_preferences

visit_preferences=femaleprobs(travel_times, nodes, lamb)


#Stay at bower ticket handler. Ticket is read, and male state changes to at bower. 
#The tickethandler can generate then generate a repair bower, foraging, or maurad travel ticket.
def SBtickethandler(SB_tic):
    ow=SB_tic['owner'] #the owner of the ticket is the male bird from the stay at bower ticket
    if male_states[ow]!=0: #because any male who is at his bower was last repairing his bower (not coming from foraging or maurading) 
        bower_states[ow]=1 #now the bower is intact, so the bower state is updated to reflect this change
    if bower_states[ow]==0: #if male returns to a destroyed bower, must repair it
        new_tic=futurebuilder(SB_tic, -7, ow) #generate a new tau and ticket with repair bower action, and add it to the timeline
    else: #if the bower is intact
        decider=random.random()
        if decider<MT_vs_FG: #transition to MT
            if strategy_states[ow]==0: #if the male is a guarder
                new_tau=SB_tic['tau']+not_maraud_time
                future_t=new_tau+SB_tic['time'] #time on future ticket will be the current time plus the tau
                if future_t>t_max: 
                    new_tau=(new_tau-(future_t-t_max)) #this ensures that the tau+t won't exceed t_max
                SB_tic['tau']=new_tau
                if new_tau > 100.0:
                    1/0
                timeline[timeholder]=SB_tic
                new_tic=futurebuilder(SB_tic, -2, ow) #generate a new tau and ticket with foraging action, and add it to the timeline
            else:
                targ=numpy.random.choice(list(range(nodes)), p=visit_preferences[ow]) #choosing the male to maurad based on visit preferences
                new_tic=futurebuilder(SB_tic, -4, targ) #generate a new tau and ticket with maurad travel action, and add it to the timeline
        else:
            new_tic=futurebuilder(SB_tic, -2, ow) #generate a new tau and ticket with foraging action, and add it to the timeline
    if new_tic=={}: 
        male_states[ow]=[t_max, -10, timeholder] #still need to show male is at bower, but -10 action is gibberish (since male won't leave bower before tmax)
    else: 
        if new_tic['action']==-7: #If the next action will be repair bower
            male_states[ow]=1 #A CHANGE: we don't know when male will leave bower but we need to state he is at bower now
        else:
            male_states[ow]=[new_tic['time'],new_tic['action'],timeholder]
            
            
            
#Foraging ticket handler. Ticket read, and male state changes to no longer at bower. Generates a stay at bower ticket.
def FGtickethandler(FG_tic):
    ow=FG_tic['owner'] 
    t=FG_tic['time']
    ta=FG_tic['tau']
    #print(male_states[ow])
    if t==0: #in this case male_states[ow]=0
        #male_states[ow]=0 #male is no longer at bower (this for if males start at SB)
        SB_tic=futurebuilder(FG_tic,-1,ow)
        if SB_tic['tau']>6:
            1/0
    elif math.isclose(male_states[ow][0],t, rel_tol=0, abs_tol=.0000001)==0:
        discards.append(FG_tic)
        timeline[timeholder]['action']=abs(FG_tic['action']) 
    else:
        male_states[ow]=0 #male is no longer at bower
        futurebuilder(FG_tic, -1, ow)
        
#Maurad travel ticket handler. Male state changes to no longer at bower, and generates a maurad action ticket.
def MTtickethandler(MT_tic):
    ow=MT_tic['owner']
    targ=MT_tic['target']
    t=MT_tic['time']
    if math.isclose(male_states[ow][0],t, rel_tol=0, abs_tol=.0000001)==0:
        discards.append(MT_tic)
        timeline[timeholder]['action']=abs(MT_tic['action']) #A CHANGE
        #print("malestates is {:f} while tictime is {:f}".format(male_states[ow][0], t))
    else:
        male_states[ow]=0 #male is no longer at his bower
        futurebuilder(MT_tic, -5, targ)

#Maurad action ticket handler. The bower destruction occurs if the targeted bower's owner is absent and the bower is intact. 
#Otherwise, the tau for this ticket is zero (bird immediately turns atruncate). This ticket handler then generates a
#maurad return ticket.
def MAtickethandler(MA_tic):
    targ=MA_tic['target']
    if male_states[targ]==0 and bower_states[targ]==1: #if the bower is intact and its owner is absent
        bower_states[targ]=0
    else: #in all other cases bowerbird immediately leaves
        MA_tic['tau']=0 #COMMENT! Changing tau for this ticket as we read it bc stay is cut short by bower owner's presence
    futurebuilder(MA_tic, -6, targ)


#Repair bower / maurad return ticket handler. This ticket handler then generates a ticket for stay at bower.
def RBMRtickethandler(RBMR_tic):
    ow=RBMR_tic['owner']
    futurebuilder(RBMR_tic, -1, ow)

       
    
#Female visitation ticket handler. Females visit and mate with males if the following conditions are met: 
#(1) The male is at the bower, (2) the bower is intact, and (3) no other female is present.
#If not,the female keeps this male on her recents list, and travels to additional males with probabilities based on a preference matrix,
#If the female finds no eligeble males but her recents list is full, a longer tau is generated. 
#Her short term memory is cleared (the recents list), and she can try any of the males again. This continues 
#until she mates succusfully. Then, the male tickets for stay at bower are extended until the female leaves. 
def FVtickethandler(FV_tic, recents_list):
    targ=FV_tic['target']
    ow=FV_tic['owner']
    t=FV_tic['time']
    recents_list=d["rl{0}".format(ow)]
    r=random.random() #generate this to decide whether mating happens if other conditions are met
    if bower_states[targ]==1 and male_states[targ]!=0 and r<success_rate and t>mating_states[targ]: #if the bower is intact and the male is present
        tau_court= 0.25 #numpy.random.normal(loc=.1504, scale=.0102, size=1)[0]
        timeline[timeholder]['tau']=tau_court #if mating is successful, then it takes .25hrs
        t_court_end=t+tau_court
        mating_states[targ]=t_court_end
        fitness_states[targ]=fitness_states[targ]+1 #assumption: female always mates if bower is intact and male present
        success_times.append(t_court_end) #this is the time at which the mating finishes (not starts, tho it could be)       
        if male_states[targ][0]<t_court_end:
            SB_tic=timeline[male_states[targ][2]] #this SB ticket has the wrong tau -- we must modify the timeline
            SB_tic['tau']=t_court_end-male_states[targ][0]+SB_tic['tau'] #the new tau is generated by tacking on the extension of SB due to FV
            timeline[male_states[targ][2]]['tau']=SB_tic['tau'] #modify the ticket on the timeline itself
            if SB_tic['tau']>50.0:
                print("ERROR at time {:f} with male {:d}".format(t, targ))
                print("t_court_end: {:f}".format(t_court_end))
                print("og time: {:f}".format(male_states[targ][0]))            
            if male_states[targ][1]==-4: #if a MT ticket, generate a new tau and new_targ (individual to be marauded by targ)
                new_targ=numpy.random.choice(list(range(nodes)), p=visit_preferences[targ]) #choosing the male to maurad based on visit preferences
            else: #if a FG ticket, generate a new tau and set new_targ to targ
                new_targ=targ #targ=ow (or in this case new_targ=targ) when it's an FG_tic
            futurebuilder(SB_tic, male_states[targ][1], new_targ)
            male_states[targ][0]=t_court_end #now change male states so that the leave_time is accurate 
    else: #if female does not successfully mate
        #HEY!!: tau is court time not time until next FV... consider changing to use futurebuilder
        recents_list.append(targ)
        if len(recents_list)==min(max_visits, nodes): #if she's already visited her max # of males and not mated, reset recents and use nexttau
            recents_list=[]
            new_FV_targ=numpy.random.choice(list(range(nodes)), p=visit_preferences[targ])
            new_FV_time=t+12 #she goes to another male after waiting some time (nexttau), courting takes no time
            if new_FV_time<t_max:
                new_FV_tic=ticketgenerator(0,new_FV_time, ow, -3, new_FV_targ, t)
                addtotimeline(new_FV_tic)
        else: #otherwise find a male not in recents and use travel_times
            new_FV_targ=-1 #just so that the code goes into the while loop the first iteration
            while(new_FV_targ in recents_list or new_FV_targ==-1):
                new_FV_targ=numpy.random.choice(list(range(nodes)), p=visit_preferences[targ]) #choose a male based on preference (a function of distance)
            new_FV_time=t+travel_times[targ][new_FV_targ] #she goes directly to this male
            if new_FV_time<t_max:
                new_FV_tic=ticketgenerator(0,new_FV_time, ow, -3, new_FV_targ, t)
                addtotimeline(new_FV_tic)
    return recents_list


[[0.         0.00231481 0.00231481 0.00327364]
 [0.00231481 0.         0.00327364 0.00231481]
 [0.00231481 0.00327364 0.         0.00231481]
 [0.00327364 0.00231481 0.00231481 0.        ]]


In [2]:
prop_marauder=.25 #proportion of marauders in population #put w parameters later
strategy_states=numpy.array([0]*nodes) #initialize strategy states, 0=guarder, 1 will = marauder
marauder_inds=numpy.random.choice(list(range(nodes)), size=int(nodes*prop_marauder), replace=False) #choose the marauders
strategy_states[marauder_inds]=1 #set these individuals to be marauders 
print(marauder_inds)

[1]


In [3]:
#random.seed(1)

sims_fitness_states=[[0.0]*nodes]*num_sims
for j in range(num_sims):
    t=0.0 #start at time 0

    timeholder=0
    timeline=[] #initialize the timeline
    discards=[] #where we put the old version of tickets that get over-written/moved
    bower_states=[1]*nodes
    male_states=[0]*nodes #start at foraging now
    mating_states=[0]*nodes
    fitness_states=[0]*nodes
    recents_list=[]

    #Seed the timeline
    d={}

    #changed so they start out foraging not SB!
    for i in range(nodes):
        init_FG_time=t+nexttau(-2, i, i, 0)
        init_FG_tic=ticketgenerator(init_FG_time-t, t, i, -2, i, {})
        addtotimeline(init_FG_tic)

    #the logic with ts and taus is not parallel to all other ticket types, but it seems to work... maybe change later    
    for i in range(int(FperM*nodes)): #FperM*nodes
        first_male=random.choice(list(range(nodes))) #The lucky bowerbird that is chosen by the female first
        first_female_time=0 #determine the time when the first female arrives
        first_female_tic=ticketgenerator(0,first_female_time, i, -3, first_male, {}) #first female ticket
        addtotimeline(first_female_tic)
        d["rl{0}".format(i)]=[]

    #read tickets, generate more tickets, and move through the timeline until t_max is reached

    while t<t_max:
        if timeholder>(len(timeline)-1): #if timeholder exceeds max timeline index
            print(j)
            break
        next_tic=timeline[timeholder] #look at the ticket corresponding to where we are on the timeline
        action=next_tic['action']
        ow=next_tic['owner']
        if action==-1:
            SBtickethandler(next_tic)
        elif action==-2:
            FGtickethandler(next_tic)
        elif action==-3:
            d["rl{0}".format(ow)]=FVtickethandler(next_tic, d["rl{0}".format(ow)])
        elif action==-4:
            MTtickethandler(next_tic)
        elif action==-5:
            MAtickethandler(next_tic)
        else:
            RBMRtickethandler(next_tic) 
        t=next_tic['time'] #new time based on the ticket we just read
        timeholder=timeholder+1 #increment timeholder
    sims_fitness_states[j]=fitness_states

        
    
 


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [4]:
d={}


for i in range(nodes):
    d["tl{0}".format(i)]=[]
for j in range(len(timeline[0:timeholder])):
    #for i in range(nodes):
     #   if timeline[j]['owner']==i:
    if timeline[j]['action']==-3:
        d["tl{0}".format(timeline[j]['target'])].append(timeline[j])  
    else:
        d["tl{0}".format(timeline[j]['owner'])].append(timeline[j])

print("tl0----")
print(d["tl0"])
#print("tl1----")
#print(d["tl1"])
#print("tl2----")
#print(d["tl2"])
#print("tl3----")
#print(d["tl3"])

tl0----
[{'tau': 0.10414255112555322, 'time': 0.0, 'owner': 0, 'action': -2, 'target': 0, 'origin': {}}, {'tau': 0, 'time': 0, 'owner': 1, 'action': -3, 'target': 0, 'origin': {}}, {'tau': 0, 'time': 0, 'owner': 4, 'action': -3, 'target': 0, 'origin': {}}, {'tau': 0, 'time': 0, 'owner': 5, 'action': -3, 'target': 0, 'origin': {}}, {'tau': 0, 'time': 0, 'owner': 10, 'action': -3, 'target': 0, 'origin': {}}, {'tau': 0, 'time': 0.0032736425054932755, 'owner': 0, 'action': -3, 'target': 0, 'origin': 0}, {'tau': 0, 'time': 0.0032736425054932755, 'owner': 3, 'action': -3, 'target': 0, 'origin': 0}, {'tau': 0, 'time': 0.005588457320308091, 'owner': 6, 'action': -3, 'target': 0, 'origin': 0.0023148148148148147}, {'tau': 0, 'time': 0.005588457320308091, 'owner': 9, 'action': -3, 'target': 0, 'origin': 0.0023148148148148147}, {'tau': 0, 'time': 0.005588457320308091, 'owner': 11, 'action': -3, 'target': 0, 'origin': 0.0032736425054932755}, {'tau': 0, 'time': 0.007903272135122905, 'owner': 2, 'act

In [5]:
acc=0
all_su=[0]*nodes
all_mar_props=[0]*nodes
all_probs=[[0]*4]*nodes
print(all_probs)
for i in range(nodes):
    tl=d["tl{0}".format(i)]
    l=len(tl)
    ta=[]
    accSB=0
    accFG=0
    accM=0
    accRB=0
    allacc=0
    FV_list=[]
    for j in range(l):
        if tl[j]['action']!=-3: #female visit taus don't mean anything for the male bc she isn't necessarily coming back to the same male
            if tl[j]['action']<0:
                ta.append(tl[j]['tau'])
            #print("time according to taus is {:f}".format(allacc))
            #print("time according to tickets is {:f}".format(tl[j]['time']))
                allacc+=tl[j]['tau']
            if tl[j]['action']==-1:
                accSB=accSB+tl[j]['tau']
                #print("action: {:d}, tau: {:f}".format(tl[j]['action'], tl[j]['tau']))
            elif tl[j]['action']==-2:
                accFG=accFG+tl[j]['tau']
                #print("action: {:d}, tau: {:f}".format(tl[j]['action'], tl[j]['tau']))
            elif tl[j]['action']==-7:
                accRB=accRB+tl[j]['tau']
                #print("action: {:d}, tau: {:f}".format(tl[j]['action'], tl[j]['tau']))
            elif tl[j]['action']==-4 or tl[j]['action']==-5 or tl[j]['action']==-6:
                accM=accM+tl[j]['tau']  
                #print("action: {:d}, tau: {:f}".format(tl[j]['action'], tl[j]['tau']))
                acc=acc+1
        else:
            #print("FV")
            FV_list.append(tl[j])
    d["taus{0}".format(i)]=ta
    su=sum([accSB/t_max, accM/t_max, accFG/t_max, accRB/t_max])
    all_su[i]=su
    print("DOES IT ADD UP: {:f}".format(su))
    print("Bird {:d}'s % time at bower is {:f}".format(i, accSB/t_max)) 
    print("Bird {:d}'s % time marauding is {:f}".format(i, accM/t_max)) 
    print("Bird {:d}'s % time foraging is {:f}".format(i, accFG/t_max)) 
    print("Bird {:d}'s % time repairing bower is {:f}".format(i, accRB/t_max)) 
    print("Bird {:d}'s separate accs sum to {:f}".format(i,(accSB+accM+accFG+accRB)))
    print("Bird {:d}'s allacc is {:f}".format(i,allacc))
    
    all_probs[i]=[accSB/t_max, accFG/t_max, accRB/t_max, accM/t_max]
    #all_probs[i][1]=accFG/t_max
    #all_probs[i][2]=accM/t_max
    #all_probs[i][3]=accRB/t_max
    all_mar_props[i]=accM/t_max

[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]
DOES IT ADD UP: 1.000000
Bird 0's % time at bower is 0.542212
Bird 0's % time marauding is 0.000000
Bird 0's % time foraging is 0.347272
Bird 0's % time repairing bower is 0.110515
Bird 0's separate accs sum to 360.000000
Bird 0's allacc is 360.000000
DOES IT ADD UP: 1.000000
Bird 1's % time at bower is 0.597783
Bird 1's % time marauding is 0.016830
Bird 1's % time foraging is 0.385387
Bird 1's % time repairing bower is 0.000000
Bird 1's separate accs sum to 360.000000
Bird 1's allacc is 360.000000
DOES IT ADD UP: 1.000000
Bird 2's % time at bower is 0.552434
Bird 2's % time marauding is 0.000000
Bird 2's % time foraging is 0.353122
Bird 2's % time repairing bower is 0.094444
Bird 2's separate accs sum to 360.000000
Bird 2's allacc is 360.000000
DOES IT ADD UP: 1.000000
Bird 3's % time at bower is 0.540141
Bird 3's % time marauding is 0.000000
Bird 3's % time foraging is 0.365415
Bird 3's % time repairing bower is 0.094444
Bird 3

In [6]:
#print(d["tl74"])

In [7]:
#print(all_probs[99])
#print(all_probs[1])

arr=numpy.array(all_probs)
print(arr[marauder_inds])
print(marauder_inds)

[[0.59778294 0.38538674 0.         0.01683032]]
[1]


In [8]:
time_spent=numpy.mean(all_mar_props)*12 #could maybe go somewhere else... justification
num_events=acc/(3*nodes)
event_time=time_spent/num_events
print(event_time) #this is the time that we'll tack on to SB... .15


0.0015902667223289184


In [9]:
#print(success_times)
print(len(success_times))

1200


In [10]:
#print(len(discards))
#print(discards)

In [11]:
#print(d["tl76"])

In [12]:
#print(sims_fitness_states)

avsfs=numpy.mean(sims_fitness_states, 0)
print(numpy.mean(avsfs[marauder_inds]))
sfs=avsfs.reshape(2,2) #HARD CODE
print(sfs) #this is spatial


3.28
[[2.99 3.28]
 [2.91 2.82]]


In [13]:
#sd/mean

In [14]:
ss=strategy_states.reshape(2,2)#HARD CODE
print(ss) #this is spatial

[[0 1]
 [0 0]]


In [15]:
print(numpy.mean(avsfs[marauder_inds]))


ia = numpy.indices(avsfs.shape)

not_indices = numpy.setxor1d(ia, marauder_inds)

print(numpy.mean(avsfs[not_indices]))

print(numpy.mean(avsfs))


3.28
2.9066666666666667
3.0
